In [28]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.optimizers import SGD # type: ignore
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping # type: ignore

In [ ]:
model_path = "/Users/michelangelozampieri/Desktop/image_classifier/models/initial_from_scratch_model_v2.h5"
train_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/train"
val_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/validation"
test_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/test"

In [13]:
model = load_model(model_path)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,454,662 (13.18 MB)

 Trainable params: 3,454,660 (13.18 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [17]:
img_height, img_width = 150, 150 
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4582 images belonging to 4 classes.
Found 1591 images belonging to 4 classes.


In [ ]:
for layer in model.layers[:-10]:  # Freeze all layers except the last 10
    layer.trainable = False 

model.compile(optimizer=SGD(learning_rate=1e-4, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [30]:
epochs = 10  

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[lr_schedule, early_stopping]
)

Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 238ms/step - accuracy: 0.9398 - loss: 0.1588 - val_accuracy: 0.9434 - val_loss: 0.1459 - learning_rate: 1.0000e-05
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 239ms/step - accuracy: 0.9416 - loss: 0.1549 - val_accuracy: 0.9459 - val_loss: 0.1431 - learning_rate: 1.0000e-05
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 35s 243ms/step - accuracy: 0.9413 - loss: 0.1529 - val_accuracy: 0.9497 - val_loss: 0.1414 - learning_rate: 1.0000e-05
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 35s 246ms/step - accuracy: 0.9415 - loss: 0.1525 - val_accuracy: 0.9491 - val_loss: 0.1392 - learning_rate: 1.0000e-05
Epoch 5/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 249ms/step - accuracy: 0.9502 - loss: 0.1349 - val_accuracy: 0.9516 - val_loss: 0.1370 - learning_rate: 1.0000e-05
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 251ms/step - accuracy: 0.9463 - loss: 0.1416 - val_accuracy: 0.9541 - val_loss: 0.1346 - learning_rate: 1.0000e-05
Epoch 7/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 25

In [31]:
model.save("fine_tuned_from_scratch.h5")

In [32]:
classes =['cloudy', 'desert', 'green_area', 'water']

def get_random_images(test_dir, num_images=5):
    random_images = []
    class_names = os.listdir(test_dir)
    for _ in range(num_images):
        random_class = random.choice(class_names)
        class_dir = os.path.join(test_dir, random_class)
        random_image = random.choice(os.listdir(class_dir))
        random_images.append((os.path.join(class_dir, random_image), random_class))
    return random_images

In [33]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array #type: ignore

def predict_random_images(model, test_dir, num_images=5):
    random_images = get_random_images(test_dir, num_images)
    for img_path, true_class in random_images:
        # Load and preprocess the image
        img = load_img(img_path, target_size=(150, 150))
        img_array = img_to_array(img) / 255.0  # Normalize the image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        
        return true_class, classes[predicted_class]

In [34]:
def evaluate_model(model, test_dir, num_images=1000):
    correct = 0
    for _ in range(num_images):
        true_class, predicted_class = predict_random_images(model, test_dir)
        if true_class == predicted_class:
            correct += 1
    return correct / num_images

In [35]:
print(evaluate_model(model, test_dir, num_images=1000))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

After fune tuning we increased the accuracy to 94.5%. This can probably be increased with more fine tuning